In [11]:
searchTerms = ["Company Name", "Address 1", "City", "State"]
key = "Company ID"
sourceFile = "workfile_small.xlsx"
indices = [17, 19]
filePath = "C:\\Users\\Spencer\\Desktop\\scrape\\py_files\\scrapes"

In [146]:
simpleScraper(sourceFile, searchTerms, key, indices=indices, wait=[1,2], combine=False, filePath=filePath)

expecting 394 reviews
scrolling...
finished scrolling, found 393 reviews
preparing to scrape location with key 1000804781
scraped location with key 1000804781 with 393 reviews
sum of all rows: 393
total rows: 393


In [148]:
driver.quit()

In [12]:
import pandas as pd
data = {'Name':['Tom', 'nick', 'krish', 'jack'], 'Age':[20, 21, 19, 18]} 
df = pd.DataFrame(data) 

In [6]:
url = "https://www.google.com/search?q=Pizza Hut+10021004 Chestnut St+EMMAUS+PA"
driver = webdriver.Chrome('./chromedriver')

In [7]:
driver.get(url)
addr = driver.find_elements_by_xpath("//span[@class='LrzXr']")

In [20]:
indices = [0,2]
age_final = [23, 25]

In [15]:
filePath[-2:]

'es'

In [19]:
filePath + "\\"

'C:\\Users\\Spencer\\Desktop\\scrape\\py_files\\scrapes\\'

In [20]:
import openpyxl
import pandas as pd

In [21]:
workfile = pd.read_excel("workfile.xlsx")

In [103]:
work_trimmed = workfile[workfile["Company ID"].notnull()]

In [86]:
import numpy as np

In [105]:
dfs = np.array_split(work_trimmed, len(work_trimmed)/500)
for df in dfs:
    first_index = str(df.first_valid_index())
    last_index = str(df.last_valid_index())
    filename = "workfile_" + first_index + "_" + last_index + ".xlsx"
    df.to_excel(filename,index=False)

In [102]:
str(dfs[0].last_valid_index())

'1291'

In [98]:
len(dfs)

10

In [107]:
# 9/29/20 3:21 scrape
# 256-306 (50 locations)
searchTerms = ["Company Name", "Address 1", "City", "State"]
key = "Company ID"
sourceFile = "workfile_530_1059.xlsx"
indices = list(range(256,307))
filePath = "C:\\Users\\Spencer\\Desktop\\scrape\\py_files\\scrapes"
# simpleScraper(sourceFile=sourceFile, searchTerms=searchTerms, key=key, indices=indices, wait=[1,2], combine=False, statusCol="Status", filePath=filePath)

In [106]:
list(range(256,530))[-1]

529

In [2]:
import pandas as pd

In [10]:
anthony = pd.read_csv("./py_files/anthony.csv")

In [13]:
df["Empty"] = ""

In [14]:
df

,Name,Age,Empty
0,Tom,20,
1,nick,21,
2,krish,19,
3,jack,18,


In [15]:
anthony["Status"]=""

In [16]:
anthony.to_csv("./py_files/anthony.csv", index=False)

In [21]:
# anthony's scrape
searchTerms = ["Name", "Street_Address", "City", "State"]
key = "Keyword"
sourceFile = "anthony.csv"
indices = list(range(0,64))
filePath = "/home/spencer/Documents/Projects/scrape/py_files/anthony_scrapes"
# failed after 40 scrapes

In [2]:
# second round of anthony scrapes
searchTerms = ["Name", "Street_Address", "City", "State"]
key = "Keyword"
sourceFile = "anthony.csv"
indices = list(range(40,64))
filePath = "/home/spencer/Documents/Projects/scrape/py_files/anthony_scrapes"
# simpleScraper(sourceFile=sourceFile, searchTerms=searchTerms,key=key,indices=indices,wait=[0.5,1.5],combine=False,statusCol="Status",filePath=filePath)

In [ ]:
# one more anthony scrape, last was successful but I missed a location
searchTerms = ["Name", "Street_Address", "City", "State"]
key = "Keyword"
sourceFile = "anthony.csv"
indices = [64]
filePath = "C:\\Users\\Spencer\\Desktop\\scrape\\py_files\\anthony_scrapes"
# simpleScraper(sourceFile=sourceFile, searchTerms=searchTerms,key=key,indices=indices,wait=[0.5,1.5],combine=False,statusCol="Status",filePath=filePath)

In [29]:
import pandas as pd
donatos = pd.read_excel("donatos.xlsx")

In [33]:
donatos["Field2"] = [" ".join(r.split(" ")[:-3]) for r in donatos["Field2"]]

In [38]:
donatos["Name"] = [" ".join(r.split(" ")[:2]) for r in donatos["Field2"]]

In [57]:
donatos["Field2"][27].split(" ")

[&#39;Donatos&#39;, &#39;Pizza&#39;, &#39;1013&#39;, &#39;High&#39;, &#39;St&#39;, &#39;HAMILTON&#39;, &#39;OH&#39;]

In [47]:
donatos["State"] = [r.split(" ")[-1] for r in donatos["Field2"]]

In [60]:
townpieces = [r.split(" ")[-3:-1] for r in donatos["Field2"]]

In [69]:
donatos["Town"] = [" ".join(t) if t[0].isupper() else t[1] for t in townpieces]

In [74]:
addrpieces = [r.split(" ")[2:-2] for r in donatos["Field2"]]

In [79]:
donatos["Address"] = [" ".join(a[0:-1]) if a[-1].isupper() else " ".join(a[0:]) for a in addrpieces]

In [82]:
donatos["Status"] = ""

In [84]:
donatos

,Field2,Unnamed: 1,Page URL,Name,Address,State,Town,Status
0,Donatos Pizza 825 W 10th St INDIANAPOLIS IN,(317) 231-9700,https://www.google.com/maps/place/Donatos+Pizz...,Donatos Pizza,825 W 10th St,IN,INDIANAPOLIS,
1,Donatos Pizza 92 S Highway 27 SOMERSET KY,(606) 677-1700,https://www.google.com/maps/place/Donatos+Pizz...,Donatos Pizza,92 S Highway 27,KY,SOMERSET,
2,Donatos Pizza 2357 E 62nd St INDIANAPOLIS IN,(317) 722-8100,https://www.google.com/maps/place/Donatos+Pizz...,Donatos Pizza,2357 E 62nd St,IN,INDIANAPOLIS,
3,Donatos Pizza 3390 Maple Ave ZANESVILLE OH,(740) 452-7900,https://www.google.com/maps/place/Donatos+Pizz...,Donatos Pizza,3390 Maple Ave,OH,ZANESVILLE,
4,Donatos Pizza 6407 Glenway Ave CINCINNATI OH,(513) 598-4200,https://www.google.com/maps/place/Donatos+Pizz...,Donatos Pizza,6407 Glenway Ave,OH,CINCINNATI,
...,...,...,...,...,...,...,...,...
150,Donatos Pizza 12515 Jefferson Ave NEWPORT NEWS VA,(757) 525-4944,https://www.google.com/maps/place/Donatos+Pizz...,Donatos Pizza,12515 Jefferson Ave,VA,NEWPORT NEWS,
151,Donatos Pizza 3464 Wales Ave NW MASSILLON OH,(330) 833-3700,https://www.google.com/maps/place/Donatos+Pizz...,Donatos Pizza,3464 Wales Ave,OH,NW MASSILLON,
152,Donatos Pizza 2601 W Parrish Ave OWENSBORO KY,(270) 855-0801,https://www.google.com/maps/place/Donatos+Pizz...,Donatos Pizza,2601 W Parrish Ave,KY,OWENSBORO,
153,Donatos Pizza 220 Azalea Square Blvd SUMMERVIL...,(843) 501-2838,https://www.google.com/maps/place/Donatos+Pizz...,Donatos Pizza,220 Azalea Square Blvd,SC,SUMMERVILLE,


In [83]:
donatos.to_csv("donatos.csv",index=False)

In [85]:
# donatos scrape
searchTerms = ["Name", "Address", "Town", "State"]
key = "Field2"
sourceFile = "donatos.csv"
indices = list(range(0,155))
filePath = "C:\\Users\\sqbrown\\Desktop\\Work\\scrape\\py_files\\scrapes"
# simpleScraper(sourceFile=sourceFile, searchTerms=searchTerms,key=key,indices=indices,wait=[0.5,1.5],combine=False,statusCol="Status",filePath=filePath)

In [ ]:
# pizza hut scrape 9/30
searchTerms = ["Company Name", "Address 1", "City", "State"]
key = "Company ID"
sourceFile = "workfile_530_1059.xlsx"
indices = list(range(257,530))
filePath = "C:\\Users\\sqbrown\\Desktop\\Work\\scrape\\py_files\\scrapes"
# simpleScraper(sourceFile=sourceFile, searchTerms=searchTerms,key=key,indices=indices,wait=[0.5,1.5],combine=False,statusCol="Status",filePath=filePath)
# successfully scraped 248

In [ ]:
# pizza hut scrape 10/1
searchTerms = ["Company Name", "Address 1", "City", "State"]
key = "Company ID"
sourceFile = "workfile_1060_1589.xlsx"
indices = list(range(0,530))
filePath = "C:\\Users\\sqbrown\\Desktop\\Work\\scrape\\py_files\\scrapes"
# simpleScraper(sourceFile=sourceFile, searchTerms=searchTerms,key=key,indices=indices,wait=[0.5,1.5],combine=False,statusCol="Status",filePath=filePath)

In [ ]:
# pizza hut scrape 10/1 #2
searchTerms = ["Company Name", "Address 1", "City", "State"]
key = "Company ID"
sourceFile = "workfile_1590_2119.xlsx"
indices = list(range(0,530))
filePath = "C:\\Users\\sqbrown\\Desktop\\Work\\scrape\\py_files\\scrapes"
# simpleScraper(sourceFile=sourceFile, searchTerms=searchTerms,key=key,indices=indices,wait=[0.5,1.5],combine=False,statusCol="Status",filePath=filePath)
# i know there is an issue with some locations not scrolling all the way. ill need to find a way to test outputs spreadsheets for this, but most seemed to work out well

In [ ]:
# pizza hut scrape 10/2 #1
searchTerms = ["Company Name", "Address 1", "City", "State"]
key = "Company ID"
sourceFile = "workfile_2120_2649.xlsx"
indices = list(range(0,530))
filePath = "C:\\Users\\sqbrown\\Desktop\\Work\\scrape\\py_files\\scrapes"
# simpleScraper(sourceFile=sourceFile, searchTerms=searchTerms,key=key,indices=indices,wait=[0.5,1.5],combine=False,statusCol="Status",filePath=filePath)

In [1]:
import pandas as pd

In [10]:
anthony = pd.read_csv(".\\py_files\\anthony_.csv")
anthony["Status"].unique()
# so, it appears all anthony's locations scraped properly
# i'll check if there are locations with data that weren't scraped
len(anthony[(anthony["Status"].isnull()) & (anthony["Keyword"].notnull())])
# many were not marked as scraped, but there are 65 locations and i have 65 scrapes, so it's fine

24

In [15]:
donatos = pd.read_csv(".\\py_files\\donatos_.csv")

In [20]:
len(donatos[donatos["Status"]=="Scraped"]) # 136, this corresponds to the 136 i have scraped
len(donatos[donatos["Status"]=="Failed"]) # 19 failed locations i will separate
len(donatos[(donatos["Status"].isnull()) & (donatos["Field2"].notnull())]) # 0 locations with no attempted scrape

0

In [23]:
# seaprate donatos failures into their own csv
donatos_f = donatos[donatos["Status"]=="Failed"]
donatos_f.to_csv("donatos_f.csv",index=False)

In [22]:
# donatos scrape to see what the issues were
searchTerms = ["Name", "Address", "Town", "State"]
key = "Field2"
sourceFile = ".\\rescrape\\donatos_f.csv"
indices = list(range(0,19))
filePath = "C:\\Users\\Spencer\\Desktop\\\scrape\\py_files\\scrapes"
# simpleScraper(sourceFile=sourceFile, searchTerms=searchTerms,key=key,indices=indices,wait=[0.5,1.5],combine=False,statusCol="Status",filePath=filePath)
# a new error came up, something about casting arrays to different sizes, trying to debug
# got it working. made a slight adjustment to how statuses are updated
# second run without address checks as they were all unneeded:
searchTerms = ["Name", "Address", "Town", "State"]
key = "Field2"
sourceFile = "donatos_f.csv"
indices = list(range(0,19))
filePath = "C:\\Users\\Spencer\\Desktop\\\scrape\\py_files\\scrapes"
# simpleScraper(sourceFile=sourceFile, searchTerms=searchTerms,key=key,indices=indices,wait=[0.5,1.5],combine=False,statusCol="Status",filePath=filePath, checkAddress=False)

19

In [1]:
import pandas as pd

In [4]:
all_stores =  pd.read_excel("C:\\Users\\Spencer\\Desktop\\\scrape\\datasets\\data.xlsx")

In [10]:
all_stores["Company Name"].unique()

array([&quot;Anthony&#39;s Coal Fired Pizza&quot;, &#39;Bananas Ultimate Juice Bar&#39;,
       &#39;Blackjack Pizza&#39;, &#39;Blaze Pizza&#39;,
       &quot;Boston&#39;s The Gourmet Pizza Restaurant &amp; Sports Bar&quot;,
       &quot;BOSTON&#39;S THE GOURMET PIZZA RESTAURANT &amp; SPORTS BAR&quot;,
       &#39;Brick Oven Pizza Company&#39;, &#39;Brixx Wood Fired Pizza&#39;,
       &#39;California Pizza Kitchen&#39;, &#39;Chanellos Pizza&#39;, &quot;Chanello&#39;s Pizza&quot;,
       &#39;CheeZies Pizza&#39;, &quot;Chuck E. Cheese&#39;s&quot;, &quot;CiCi&#39;s Pizza&quot;,
       &quot;CiCi&#39;s To Go&quot;, &#39;Cobalt&#39;, &#39;Cottage Inn Pizza&#39;, &#39;CPK ASAP&#39;,
       &quot;D&#39;Angelo Grilled Sandwiches&quot;, &#39;Deweys Pizza&#39;, &quot;Domino&#39;s&quot;,
       &#39;Donatos Pizza&#39;, &#39;Extreme Pizza&#39;, &quot;Figaro&#39;s Pizza&quot;,
       &quot;Fox&#39;s Pizza Den&quot;, &#39;GattiLand&#39;, &quot;Gatti&#39;s Pizza&quot;,
       &quot;Gatti&#39;s Pizza

In [9]:
cpk = all_stores[all_stores["Company Name"] == "California Pizza Kitchen"]
cpk.to_csv("cpk.csv" ,index=False)

In [11]:
chuckecheese = all_stores[all_stores["Company Name"] == "Chuck E. Cheese's"]
chuckecheese.to_csv("chuckecheese.csv" ,index=False)

In [12]:
pizzaranch = all_stores[all_stores["Company Name"] == "The Pizza Ranch"]
pizzaranch.to_csv("pizzaranch.csv" ,index=False)

In [14]:
print(len(cpk))
print(len(chuckecheese))
print(len(pizzaranch))

192
531
207
